In [1]:
import jax
import jax.numpy as np
import jraph
import flax.linen as nn

from functools import partial

In [2]:
%load_ext autoreload
%autoreload 2

In [24]:
import sys
sys.path.append("../")

from models.gnn import GraphConvNet
from models.graph_utils import nearest_neighbors

In [25]:
x = np.load("data/halos_small.npy")
n_nodes = 5000

sources, targets, dist = jax.vmap(nearest_neighbors, in_axes=(0, None))(x[:4], 50)

In [26]:
graph = jraph.GraphsTuple(
          n_node=np.array(4 * [[n_nodes]]), 
          n_edge=np.array(4 * [[50]]),
          nodes=x[:4], 
          edges=None,#np.zeros(sources.shape),
          globals=np.ones((4, 7)),
          senders=sources,
          receivers=targets)

In [27]:
graph.n_node.shape, graph.n_edge.shape, graph.nodes.shape, graph.globals.shape, graph.senders.shape, graph.receivers.shape, 

((4, 1), (4, 1), (4, 5000, 7), (4, 7), (4, 250000), (4, 250000))

In [32]:
class GraphWrapper(nn.Module):
    @nn.compact
    def __call__(self, x):
        model = jax.vmap(GraphConvNet(
            latent_size=6, 
            hidden_size=32,
            num_mlp_layers=4, 
            attention=True,
            num_heads=4,
            message_passing_steps=3, 
            shared_weights=False,
            skip_connections=True,))
        return model(x)

In [33]:
model = GraphWrapper()
rng = jax.random.PRNGKey(42)
graph_out, _ = model.init_with_output(rng, graph)

In [34]:
graph_out.nodes.shape

(4, 5000, 3)